In [57]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse
import seaborn as sns
from scipy import spatial
from scipy.sparse import coo_matrix
from sklearn.decomposition import PCA
from tqdm.auto import tqdm

In [58]:
THREADS = 8

In [ ]:
epochs = [5, 25, 50, 250, 500, 1000, 1500]
models = [pd.read_pickle(f'/pio/scratch/1/i313924/data/lightfm_data/model_{epochs}_epochs_all_clusters_rho_0.8.pkl') for epochs in epochs]
train_interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/5_core_interactions.npz')
test_interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/5_core_test_interactions.npz')

In [ ]:
dataset = pd.read_pickle('/pio/scratch/1/i313924/data/lightfm_data/5_core_dataset.pkl')

## Utils

In [ ]:
def hit_rate(results_pdf, k=50):
    return np.average(
        results_pdf.groupby('reviewerID').progress_apply(
            lambda pdf: np.any(pdf['rank'] <= k)
        )
    )

def precision(results_pdf, k=50):
    return np.average(
        results_pdf.groupby('reviewerID').progress_apply(
            lambda pdf: np.sum(pdf['rank'] <= k) / k
        )
    )

def recall(results_pdf, k=50):
    return np.average(
        results_pdf.groupby('reviewerID').progress_apply(
            lambda pdf: np.sum(pdf['rank'] <= k) / len(pdf)
        )
    )

In [ ]:
def calculate_metrics(results, user_mapping, item_mapping):
    results = coo_matrix(results)
    results_pdf = pd.DataFrame(
        np.vstack((results.row, results.col, results.data)).T,
        columns=['reviewerID', 'asin', 'rank'],
        dtype=int
    )
    results_pdf['reviewerID'] = results_pdf['reviewerID'].map(user_mapping)
    results_pdf['asin'] = results_pdf['asin'].map(item_mapping)

    return hit_rate(results_pdf), recall(results_pdf), precision(results_pdf)

In [ ]:
def analyse_metrics(models, train_interactions, test_interactions, dataset):

    test_precision_scores = []
    test_recall_scores = []
    test_hit_rate_scores = []

    item_mapping = {v: k for k, v in dataset.mapping()[2].items()}
    user_mapping = {v: k for k, v in dataset.mapping()[0].items()}
    for model in tqdm(models):

        results = model.predict_rank(test_interactions, train_interactions, num_threads=8)
        hr, r, p = calculate_metrics(results, user_mapping, item_mapping)

        test_hit_rate_scores.append(hr)
        test_recall_scores.append(r)
        test_precision_scores.append(p)

    return test_hit_rate_scores, test_recall_scores, test_precision_scores

In [ ]:
def analyse_ABX(path, items_embeddings, pca):
    abx_tests = pd.read_json(path, lines=True)
    A = np.zeros((10000, 100))
    B = np.zeros((10000, 100))
    X = np.zeros((10000, 100))

    for i, test in abx_tests.iterrows():
        A[i, :] = items_embeddings.loc[test["A"]]
        B[i, :] = items_embeddings.loc[test["B"]]
        X[i, :] = items_embeddings.loc[test["X"]]

    dist_A = ((A - X)**2).sum(axis=1)
    dist_B = ((B - X)**2).sum(axis=1)

    cos_dist_A = np.zeros(10000)
    cos_dist_B = np.zeros(10000)

    for i in range(10000):
        cos_dist_A[i] = spatial.distance.cosine(A[i, :], X[i, :])
        cos_dist_B[i] = spatial.distance.cosine(B[i, :], X[i, :])

    if pca is not None:
        pca_A = pca.transform(A)
        pca_B = pca.transform(B)
        pca_X = pca.transform(X)

        dist_pca_A = ((pca_A - pca_X)**2).sum(axis=1)
        dist_pca_B  = ((pca_B - pca_X)**2).sum(axis=1)

        cos_dist_pca_A = np.zeros(10000)
        cos_dist_pca_B = np.zeros(10000)

        for i in range(10000):
            cos_dist_pca_A[i] = spatial.distance.cosine(pca_A[i, :], pca_X[i, :])
            cos_dist_pca_B[i] = spatial.distance.cosine(pca_B[i, :], pca_X[i, :])

    return [(dist_A < dist_B).mean(), (dist_pca_A < dist_pca_B).mean(), ((dist_A < dist_B) == (dist_pca_A < dist_pca_B)).mean()], [(cos_dist_A < cos_dist_B).mean(), (cos_dist_pca_A < cos_dist_pca_B).mean(), ((cos_dist_A < cos_dist_B) == (cos_dist_pca_A < cos_dist_pca_B)).mean()]

In [ ]:
def analyse(num_epochs, models, categories, women_ids, men_ids):

    item_embeddings = pd.DataFrame()
    user_embeddings = pd.DataFrame()

    for model in models:

        item_embeddings = pd.concat([item_embeddings, pd.DataFrame(model.item_embeddings)])
        user_embeddings = pd.concat([user_embeddings, pd.DataFrame(model.user_embeddings)])

    item_pca = PCA(n_components=2)
    item_pca.fit(item_embeddings)
    user_pca = PCA(n_components=2)
    user_pca.fit(user_embeddings)

    print("PCA prepared")

    eucl_scores = []
    cos_scores = []
    women_means = []
    men_means = []

    for i, (epochs, model) in enumerate(zip(num_epochs, models)):

        figure, axis = plt.subplots(1, 2, figsize=(12,6))
        print(f"EPOCH: {epochs}")

        item_mapping = {v: k for k, v in dataset.mapping()[2].items()}

        item_embeddings = pd.DataFrame(model.item_embeddings)
        user_embeddings = pd.DataFrame(model.user_embeddings)

        reduced_item_embeddings = pd.DataFrame(item_pca.transform(item_embeddings))
        reduced_user_embeddings = pd.DataFrame(user_pca.transform(user_embeddings))

        axis[0].scatter(reduced_user_embeddings[0], reduced_user_embeddings[1], s=0.03)
        axis[0].set_xlim([-1, 1])
        axis[0].set_ylim([-1, 1])

        reduced_item_embeddings = reduced_item_embeddings.rename(index=item_mapping)

        reduced_item_embeddings = reduced_item_embeddings.join(categories.set_index('asin'))
        reduced_item_embeddings.columns = ['0', '1', 'category_1', 'category_2']

        sns.scatterplot(x='0', y='1', data=reduced_item_embeddings, hue='category_1', s=10, ax=axis[1], legend=False)
        axis[1].set_xlim([-5, 5])
        axis[1].set_ylim([-5, 5])
        figure.show()

        item_embeddings = item_embeddings.rename(index=item_mapping)

        women_mean = item_embeddings.loc[women_ids].mean(axis = 0)
        men_mean = item_embeddings.loc[men_ids].mean(axis = 0)

        women_means.append(women_mean)
        men_means.append(men_mean)

        eucl_score, cos_score = analyse_ABX('/pio/scratch/1/recommender_systems/interim/ABX_tests/5_core.json', item_embeddings, pca = item_pca)

        eucl_scores.append(eucl_score)
        cos_scores.append(cos_score)

    return eucl_scores, cos_scores, women_means, men_means, item_pca

In [ ]:
categories = pd.read_json('/pio/scratch/1/recommender_systems/interim/Amazon/meta_Clothing_Shoes_and_Jewelry_categories.json', lines=True)

In [ ]:
item_ids = list(dataset.mapping()[2].keys())
women_ids = categories[(categories.category_1 == "Women") & (categories.asin.isin(item_ids))].asin.values
men_ids = categories[(categories.category_1 == "Men") & (categories.asin.isin(item_ids))].asin.values

In [ ]:
e, c, women_means, men_means, item_pca = analyse(epochs, models, categories, women_ids, men_ids)

In [ ]:
print("Eucl dist")
plt.plot(np.tile(epochs, (3, 1)).T, e)
plt.legend(["score", "pca score", "pca dist match"])
plt.show()
print("Cos dist")
plt.plot(np.tile(epochs, (3, 1)).T, c)
plt.legend(["score", "pca score", "pca dist match"])
plt.show()

In [ ]:
e

In [ ]:
c

In [ ]:
tqdm.pandas()
test_hit_rate_scores, test_recall_scores, test_precision_scores = analyse_metrics(models, train_interactions, test_interactions, dataset)

In [ ]:
test_recall_scores

In [ ]:
test_precision_scores

In [ ]:
test_hit_rate_scores

In [ ]:
model = models[-1]
item_embeddings = pd.DataFrame(model.item_embeddings)
item_mapping = {v: k for k, v in dataset.mapping()[2].items()}
pca = PCA(n_components=2)
pca.fit(item_embeddings)
reduced_item_embeddings = pd.DataFrame(pca.transform(item_embeddings))
reduced_item_embeddings = reduced_item_embeddings.rename(index=item_mapping)
reduced_item_embeddings = reduced_item_embeddings.join(categories.set_index('asin'))
reduced_item_embeddings.columns = ['0', '1', 'category_1', 'category_2']

In [ ]:
reduced_item_embeddings.category_1.value_counts()

In [ ]:
sns.scatterplot(x='0', y='1', data=reduced_item_embeddings[~reduced_item_embeddings['category_1'].isin(['Women', 'Men'])], hue='category_1', s=10, legend=False)

In [ ]:
sns.scatterplot(x='0', y='1', data=reduced_item_embeddings[reduced_item_embeddings['category_1'].isin(['Boys', 'Girls', 'Baby'])], hue='category_1', s=10, legend=True)

In [ ]:
sns.scatterplot(x='0', y='1', data=reduced_item_embeddings[reduced_item_embeddings['category_1'].isin(['Boys', 'Women'])], hue='category_1', s=10, legend=True)

In [ ]:
sns.scatterplot(x='0', y='1', data=reduced_item_embeddings[reduced_item_embeddings['category_1'].isin(['Boys', 'Men'])], hue='category_1', s=10, legend=True)

In [ ]:
sns.scatterplot(x='0', y='1', data=reduced_item_embeddings[reduced_item_embeddings['category_1'].isin(['Novelty & More', 'Men'])], hue='category_1', s=10, legend=True)

In [ ]:
sns.scatterplot(x='0', y='1', data=reduced_item_embeddings[reduced_item_embeddings['category_1'].isin(['Novelty & More', 'Boys'])], hue='category_1', s=10, legend=True)

In [ ]:
sns.scatterplot(x='0', y='1', data=reduced_item_embeddings[reduced_item_embeddings['category_1'].isin(['Novelty & More', 'Girls'])], hue='category_1', s=10, legend=True)